In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
%reload_ext autoreload

In [30]:
# logging 
import logging
import logging.config

# parse configuration files
import configparser

# parse command line arguments
import argparse

# handle file paths in a sane way
from pathlib import Path

In [31]:
# this works best as a global variable
# FIXME - best practice for specifying this?
logConfig = Path('./logging.cfg')
logging.config.fileConfig(logConfig.absolute())
# logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(name)s %(levelname)s: %(message)s')

logger = logging.getLogger(__name__)

In [32]:
def arg_parse():
    parser = argparse.ArgumentParser()
    
    # configuration file
    parser.add_argument('-c', type=str, required=False,
                        help='use the specified configuration file. Default is stored in ~/.config/myApp/config.ini')
    
    
    
    args, unknown = parser.parse_known_args()
    logging.info(f'discarding unknwon commandline arguments: {unknown}')
    
    return args

In [33]:
def read_config(cfgfile=None, required={}):
    if not cfgfile:
        return {}
    
    config = configparser.ConfigParser()
    logging.info(f'reading config file {cfgfile}')
    config.read(cfgfile)
    
    try:
        for section in required:
            if section not in config.sections():
                logging.debug(f'adding missing section: {section}')
                config.add_section(section)
        
            for option in required[section]:
                if not config.has_option(section, option):
                    logger.debug(f'adding missing option: {option}')
                    config[section][option] = str(required[section][option])
            with open(cfgfile, 'w') as file:
                config.write(file)
    except Exception as e:
        logging.exception(f'exception while writing config file: {e}')
        raise
    
    return config

In [10]:
import sys

In [11]:
sys.argv.append('-c')
sys.argv.append('~/slimpi.ini')

In [21]:
args = arg_parse()

In [35]:
cfg = read_config(args.c, required)

<ipython-input-33-230504a7eb23>:read_config:6:INFO - reading config file ~/slimpi.ini


In [34]:
cfg.sections()

[]

In [ ]:
def main():
    # Configuration Variables
    # Default configuration file if none is specified
    default_cfg = Path('./slimpi.cfg')
    
    